In [1]:
import argparse
from datetime import datetime
from functools import partial
import itertools
import time
import os
import pickle

import numpy as np
import matplotlib.pyplot as plt

from src.qm_utils.lattice.lattice import Lattice2D
from src.netket_compat import get_sector_constraints
from src.acband import acband_form_factors, K_func1, K_func2

from brillouin_zones import construct_brillouin_zones

/home/cwpark/acband-laughlin-instability/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


∣NK⟩ Tip: You must cite NetKet according to our policy. Use nk.cite() to find out how.

In [2]:
N_s = 27
K = 0.4

sqrt3 = np.sqrt(3)

lB = 1.0
a_M = (((4 * np.pi) / sqrt3) ** 0.5) * lB

# fourier_resolution = 128
fourier_resolution = 256
G_radius = 64
V1 = 1.0
v1 = 3 * V1 * (a_M ** 4) / (4 * np.pi)

e1 = np.array([1, 0])
e2 = np.array([0, 1])
a1 = a_M * e2
a2 = a_M * ((-sqrt3 / 2) * e1 + (1 / 2) * e2)
lattice = Lattice2D(np.stack([a1, a2]))
recip_lattice = lattice.reciprocal()

bz = construct_brillouin_zones(lattice)

bz_N_s = bz[N_s]

A = N_s * lattice.unit_cell_area

def V(q):
    return -v1 * np.linalg.norm(q, axis=-1) ** 2
 
b1, b2 = lattice.reciprocal_lattice_vectors
b3 = -(b1 + b2)

K_func_args = (K, b1, b2, b3)
K_func = partial(K_func1, args=K_func_args)


start = time.time()
G_coords, ac_ff = acband_form_factors(
    bz[N_s],
    lB,
    K_func,
    fourier_resolution,
    G_radius=G_radius,
    pbar=True
)
end = time.time()

Computing AC band form factors: 100%|██████████| 16641/16641 [08:36<00:00, 32.22it/s]


In [3]:
%load_ext juliacall

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [4]:
%%julia

include("julia_ref/code_00_Lattice.jl")
include("julia_ref/code_01_FiniteLattice.jl")
include("julia_ref/code_10_InteractionMatrix.jl")
include("julia_ref/code_20_ManyBodyHamiltonian.jl")

ManyBodyHamiltonian_MK_2V1B_2B_col (generic function with 1 method)

In [ ]:
asdf = 1.0 + 0.0j

In [5]:
%%julia

println($bz_N_s.n11)

JuliaError: syntax: "$" expression outside quote
Stacktrace:
 [1] top-level scope
   @ none:1
 [2] eval
   @ ./boot.jl:430 [inlined]
 [3] eval
   @ ./Base.jl:130 [inlined]
 [4] pyjlmodule_seval(self::Module, expr::Py)
   @ PythonCall.JlWrap ~/.julia/packages/PythonCall/mkWc2/src/JlWrap/module.jl:13
 [5] _pyjl_callmethod(f::Any, self_::Ptr{PythonCall.C.PyObject}, args_::Ptr{PythonCall.C.PyObject}, nargs::Int64)
   @ PythonCall.JlWrap ~/.julia/packages/PythonCall/mkWc2/src/JlWrap/base.jl:65
 [6] _pyjl_callmethod(o::Ptr{PythonCall.C.PyObject}, args::Ptr{PythonCall.C.PyObject})
   @ PythonCall.JlWrap.Cjl ~/.julia/packages/PythonCall/mkWc2/src/JlWrap/C.jl:63

In [ ]:
%%julia

fl = TriangularLattice()
